### Getting the coordinates of Central New York

In [1]:
!pip install geopy

Waiting for a Spark session to start...
Spark Initialization Done! ApplicationId = app-20191123200342-0000
KERNEL_ID = fd0c072f-2818-4e27-81b1-2128cf8a5b73
    100% |################################| 102kB 2.8MB/s a 0:00:01


In [2]:
!pip install folium

    100% |################################| 92kB 3.1MB/s eta 0:00:01
    100% |################################| 61kB 2.0MB/s eta 0:00:01
    100% |################################| 133kB 2.5MB/s eta 0:00:01
    100% |################################| 20.0MB 596kB/s eta 0:00:01
    100% |################################| 61kB 1.8MB/s eta 0:00:01
    100% |################################| 163kB 3.6MB/s eta 0:00:01
    100% |################################| 133kB 3.1MB/s eta 0:00:01
    100% |################################| 143kB 3.5MB/s eta 0:00:01
tensorflow 1.13.1 requires tensorboard<1.14.0,>=1.13.0, which is not installed.
ibm-cos-sdk-core 2.4.3 has requirement urllib3<1.25,>=1.20, but you'll have urllib3 1.25.7 which is incompatible.
botocore 1.12.82 has requirement urllib3<1.25,>=1.20, but you'll have urllib3 1.25.7 which is incompatible.


In [3]:
import requests # library to handle requests
import pandas as pd # library for data analsysis
import numpy as np # library to handle data in a vectorized manner
import random # library for random number generation

!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values

# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 
    
# tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize

!conda install -c conda-forge folium=0.5.0 --yes
import folium # plotting library


Solving environment: done

## Package Plan ##

  environment location: /home/spark/shared/conda/envs/python3.6

  added / updated specs:
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    ca-certificates-2019.9.11  |       hecc5488_0         144 KB  conda-forge
    certifi-2019.9.11          |           py36_0         147 KB  conda-forge
    geographiclib-1.50         |             py_0          34 KB  conda-forge
    geopy-1.20.0               |             py_0          57 KB  conda-forge
    openssl-1.1.1d             |       h516909a_0         2.1 MB  conda-forge
    ------------------------------------------------------------
                                           Total:         2.5 MB

The following NEW packages will be INSTALLED:

  geographiclib      conda-forge/noarch::geographiclib-1.50-py_0
  geopy              conda-forge/noarch::geopy-1.20.0-py_0

The followin

toolz-0.10.0         | 46 KB     | ##################################### | 100% 
branca-0.3.1         | 25 KB     | ##################################### | 100% 
urllib3-1.25.7       | 160 KB    | ##################################### | 100% 
pandas-0.25.3        | 11.4 MB   | ##################################### | 100% 
chardet-3.0.4        | 190 KB    | ##################################### | 100% 
pycparser-2.19       | 173 KB    | ##################################### | 100% 
liblapack-3.8.0      | 10 KB     | ##################################### | 100% 
libopenblas-0.3.7    | 7.6 MB    | ##################################### | 100% 
libblas-3.8.0        | 10 KB     | ##################################### | 100% 
python-dateutil-2.8. | 220 KB    | ##################################### | 100% 
libgfortran-ng-7.3.0 | 1.7 MB    | ##################################### | 100% 
entrypoints-0.3      | 12 KB     | ##################################### | 100% 
pyopenssl-19.0.0     | 81 KB

In [4]:
address = 'New York central, NY'

geolocator = Nominatim()
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude

/opt/ibm/conda/miniconda3.6/lib/python3.6/site-packages/ipykernel/__main__.py:3: DeprecationWarning: Using Nominatim with the default "geopy/1.20.0" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  app.launch_new_instance()


### Get the list of hotels within 5 km range of the location, limit 50

#### Foursquare credentials

In [5]:
CLIENT_ID = 'A54AYVKDDOYWVQTJMVEEB3NI10XTR3VZU0RJQGF5QL1ZYB1C' # your Foursquare ID
CLIENT_SECRET = 'IIINL2WE1LYT0RX2SC4455CASV0HFS5V5MLVIN0OX43WDTPH' # your Foursquare Secret
VERSION = '20180604'
LIMIT = 20

#### list of hotels within 5 km of central New York location

In [6]:
search_query = 'Hotel'
radius = 5000
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
results = requests.get(url).json()
venues = results['response']['venues']
dataframe = json_normalize(venues)

#### extracting relevant columns only

In [7]:
filtered_columns = ['name', 'categories'] + [col for col in dataframe.columns if col.startswith('location.')] + ['id']
dataframe_filtered = dataframe.loc[:, filtered_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
dataframe_filtered['categories'] = dataframe_filtered.apply(get_category_type, axis=1)

# clean column names by keeping only last term
dataframe_filtered.columns = [column.split('.')[-1] for column in dataframe_filtered.columns]

#### remove rows having category other than Hotel

In [8]:
df_filtered = dataframe_filtered[dataframe_filtered['categories'] == 'Hotel']

In [9]:
df_final = df_filtered.reset_index()
df_final

,index,name,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,postalCode,state,id
0,0,The Roxy Hotel,Hotel,2 Avenue of the Americas,US,New York,United States,btwn Walker & White St,740,[2 Avenue of the Americas (btwn Walker & White...,"[{'label': 'display', 'lat': 40.71934055853672...",40.719341,-74.005044,10013,NY,3fd66200f964a520bbe61ee3
1,1,Smyth Hotel,Hotel,85 W Broadway,US,New York,United States,Chambers St,379,"[85 W Broadway (Chambers St), New York, NY 100...","[{'label': 'display', 'lat': 40.7151439, 'lng'...",40.715144,-74.009183,10007,NY,49efcc88f964a52006691fe3
2,2,Four Seasons Hotel New York Downtown,Hotel,27 Barclay St,US,New York,United States,Broadway,284,"[27 Barclay St (Broadway), New York, NY 10007,...","[{'label': 'display', 'lat': 40.71261246501983...",40.712612,-74.009380,10007,NY,57c640ad498e74977f98372f
3,3,Greenwich Hotel,Hotel,377 Greenwich St,US,New York,United States,N. Moore,862,"[377 Greenwich St (N. Moore), New York, NY 100...","[{'label': 'display', 'lat': 40.71987580077512...",40.719876,-74.009949,10013,NY,4ab6c985f964a5200a7920e3
4,4,Hotel 50 Bowery NYC,Hotel,50 Bowery,US,New York,United States,btwn Bayard & Canal St,856,"[50 Bowery (btwn Bayard & Canal St), New York,...","[{'label': 'display', 'lat': 40.7159364, 'lng'...",40.715936,-73.996789,10013,NY,578692f4498e1054905dbde7
5,5,Walker Hotel Tribeca,Hotel,396 Broadway,US,New York,United States,Walker Street,706,"[396 Broadway (Walker Street), New York, NY 10...","[{'label': 'display', 'lat': 40.71848123880058...",40.718481,-74.002469,10013,NY,5648ead638fa50411e04d842
6,6,Soho Grand Hotel,Hotel,54 Watts St,US,New York,United States,Between Grand Street & Canal Street,1246,[54 Watts St (Between Grand Street & Canal Str...,"[{'label': 'display', 'lat': 40.72391073, 'lng...",40.723911,-74.005224,10013,NY,3fd66200f964a52053eb1ee3
7,7,Hotel Indigo,Hotel,171 Ludlow St,US,New York,United States,btw Houston & Stanton,1816,"[171 Ludlow St (btw Houston & Stanton), New Yo...","[{'label': 'display', 'lat': 40.72176221543275...",40.721762,-73.988092,10002,NY,54c0151d498e4c827296cd41
8,8,The Beekman - A Thompson Hotel,Hotel,123 Nassau St,US,New York,United States,at Beekman St,182,"[123 Nassau St (at Beekman St), New York, NY 1...","[{'label': 'display', 'lat': 40.7111725, 'lng'...",40.711173,-74.006702,10038,NY,56d8c0f8498edb854f926e6a
9,9,Mercer Hotel,Hotel,147 Mercer St,US,New York,United States,at Prince St,1486,"[147 Mercer St (at Prince St), New York, NY 10...","[{'label': 'display', 'lat': 40.72482768273195...",40.724828,-73.998553,10012,NY,4a32f2a7f964a520cf9a1fe3


#### exploring the selected hotel

In [9]:
venue_id = df_final.iloc[2,15] 
url = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&v={}'.format(venue_id, CLIENT_ID, CLIENT_SECRET, VERSION)
result = requests.get(url).json()

In [10]:
venues1= result['response']['venue']
df = json_normalize(venues1)
df

,allowMenuUrlEdit,attributes.groups,beenHere.count,beenHere.lastCheckinExpiredAt,beenHere.marked,beenHere.unconfirmedCount,bestPhoto.createdAt,bestPhoto.height,bestPhoto.id,bestPhoto.prefix,...,reasons.items,shortUrl,specials.count,specials.items,stats.tipCount,timeZone,tips.count,tips.groups,url,verified
0,True,"[{'type': 'payments', 'name': 'Credit Cards', ...",0,0,False,0,1561663842,1920,5d151962b990ec002366c6df,https://fastly.4sqi.net/img/general/,...,"[{'summary': 'Lots of people like this place',...",http://4sq.com/2bYZf8E,0,[],4,America/New_York,4,"[{'type': 'others', 'name': 'All tips', 'count...",https://www.fourseasons.com/newyork/,True


#### extracting relevant columns only

In [13]:
filtered_columns = ['name', 'categories'] + [col for col in df.columns if col.startswith('location.')] + ['id']
df_filtered = df.loc[:, filtered_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
df_filtered['categories'] = df_filtered.apply(get_category_type, axis=1)

# clean column names by keeping only last term
df_filtered.columns = [column.split('.')[-1] for column in df_filtered.columns]

#### filter hotels based on required amenities

In [14]:
df_filtered

,name,categories,address,cc,city,country,crossStreet,formattedAddress,labeledLatLngs,lat,lng,postalCode,state,id
0,Four Seasons Hotel New York Downtown,Hotel,27 Barclay St,US,New York,United States,Broadway,"[27 Barclay St (Broadway), New York, NY 10007,...","[{'label': 'display', 'lat': 40.71261246501983...",40.712612,-74.00938,10007,NY,57c640ad498e74977f98372f


#### Selecting 40 most famous venues near the selected hotel

In [12]:
hotel_lat = df_final.iloc[2,11]
hotel_lon = df_final.iloc[2,12]
#hotel_lat
#hotel_lon

In [26]:
radius = 20000
LIMIT = 40
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, VERSION, hotel_lat, hotel_lon, radius, LIMIT)
results = requests.get(url).json()
#results

In [27]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [28]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng', 'venue.location.distance' ]
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues

,name,categories,lat,lng,distance
0,The Bar Room at Temple Court,Hotel Bar,40.711448,-74.006802,253
1,9/11 Memorial North Pool,Memorial Site,40.712077,-74.013187,326
2,One World Trade Center,Building,40.713069,-74.013133,320
3,9/11 Memorial South Pool,Memorial Site,40.711039,-74.013082,358
4,Washington Market Park,Playground,40.717046,-74.011095,514
5,Aire Ancient Baths,Spa,40.718141,-74.004941,720
6,Crown Shy,Restaurant,40.706187,-74.007490,732
7,Battery Park City Esplanade,Park,40.711622,-74.017907,727
8,Nelson A. Rockefeller Park,Park,40.717095,-74.016716,795
9,sweetgreen,Salad Place,40.705586,-74.008382,786


#### visualizing the hotel and venues on map

In [47]:
venues_map = folium.Map(location=[latitude, longitude], zoom_start=15) # generate map centred around Ecco


# add Ecco as a red circle mark
folium.features.CircleMarker(
    [latitude, longitude],
    radius=10,
    popup='Ecco',
    fill=True,
    color='red',
    fill_color='red',
    fill_opacity=0.6
    ).add_to(venues_map)


# add popular spots to the map as blue circle markers
for lat, lng, label in zip(nearby_venues.lat, nearby_venues.lng, nearby_venues.categories):
    folium.features.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        fill=True,
        color='blue',
        fill_color='blue',
        fill_opacity=0.6
        ).add_to(venues_map)

# display map
venues_map

#### cluster analysis of selected venues based on distance from the hotel

In [44]:
from sklearn.cluster import KMeans
import matplotlib.cm as cm
import matplotlib.colors as colors

In [40]:
kclusters = 5

nearby_venues_clustering = nearby_venues.drop(['name', 'categories'],1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(nearby_venues_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_

array([3, 3, 3, 3, 3, 1, 1, 1, 1, 1, 1, 4, 4, 4, 4, 2, 2, 2, 2, 2, 2, 2,
       2, 0, 2, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=int32)

In [41]:
nearby_venues.insert(0, 'Cluster Labels', kmeans.labels_)
#nearby_venues

,Cluster Labels,name,categories,lat,lng,distance
0,3,The Bar Room at Temple Court,Hotel Bar,40.711448,-74.006802,253
1,3,9/11 Memorial North Pool,Memorial Site,40.712077,-74.013187,326
2,3,One World Trade Center,Building,40.713069,-74.013133,320
3,3,9/11 Memorial South Pool,Memorial Site,40.711039,-74.013082,358
4,3,Washington Market Park,Playground,40.717046,-74.011095,514
5,1,Aire Ancient Baths,Spa,40.718141,-74.004941,720
6,1,Crown Shy,Restaurant,40.706187,-74.007490,732
7,1,Battery Park City Esplanade,Park,40.711622,-74.017907,727
8,1,Nelson A. Rockefeller Park,Park,40.717095,-74.016716,795
9,1,sweetgreen,Salad Place,40.705586,-74.008382,786


In [49]:
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(nearby_venues['lat'], nearby_venues['lng'], nearby_venues['name'], nearby_venues['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.9).add_to(map_clusters)
       
map_clusters

#### 5 most famous Indian restaurants near the hotel

In [12]:
search_query = 'Indian Restaurant'
radius = 1000
LIMIT = 15
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, hotel_lat, hotel_lon, VERSION, search_query, radius, LIMIT)
results = requests.get(url).json()
venues = results['response']['venues']
dataframe = json_normalize(venues)
#dataframe

,categories,delivery.id,delivery.provider.icon.name,delivery.provider.icon.prefix,delivery.provider.icon.sizes,delivery.provider.name,delivery.url,hasPerk,id,location.address,...,location.distance,location.formattedAddress,location.labeledLatLngs,location.lat,location.lng,location.postalCode,location.state,name,referralId,venuePage.id
0,"[{'id': '4bf58dd8d48988d10f941735', 'name': 'I...",263979,/delivery_provider_seamless_20180129.png,https://fastly.4sqi.net/img/general/cap/,"[40, 50]",seamless,https://www.seamless.com/menu/bombays-indian-r...,False,4c17b29b838020a1fd0de361,60 Pearl St,...,1017,"[60 Pearl St (Broad St.), New York, NY 10004, ...","[{'label': 'display', 'lat': 40.70356930142193...",40.703569,-74.011141,10004,NY,Bombay's Indian Restaurant,v-1574822654,93243558
1,"[{'id': '4bf58dd8d48988d115941735', 'name': 'M...",NaN,NaN,NaN,NaN,NaN,NaN,False,4c4890da1879c9b6cce6e143,183 Church St,...,397,"[183 Church St (btwn Duane & Reade St.), New Y...","[{'label': 'display', 'lat': 40.71578949233164...",40.715789,-74.007227,10007,NY,New Shezan Restaurant,v-1574822654,NaN
2,"[{'id': '4bf58dd8d48988d1cb941735', 'name': 'F...",NaN,NaN,NaN,NaN,NaN,NaN,False,5072fe2eb0ed0fcce2497f30,Water St,...,282,"[Water St (btwn Maiden Lane & Pine St), New Yo...","[{'label': 'display', 'lat': 40.71458096357580...",40.714581,-74.011493,10005,NY,Indian Biryani Delights (Cart),v-1574822654,NaN
3,"[{'id': '4bf58dd8d48988d11b941735', 'name': 'P...",NaN,NaN,NaN,NaN,NaN,NaN,False,49f125dcf964a52091691fe3,120 Cedar St,...,420,"[120 Cedar St (at Greenwich St.), New York, NY...","[{'label': 'display', 'lat': 40.70989378141622...",40.709894,-74.012836,10006,NY,O'Hara's Restaurant & Pub,v-1574822654,93216281
4,"[{'id': '4bf58dd8d48988d10f941735', 'name': 'I...",NaN,NaN,NaN,NaN,NaN,NaN,False,4bbb9dbded7776b0e1ad3e51,99 Hudson St,...,736,"[99 Hudson St (at Franklin St), New York, NY 1...","[{'label': 'display', 'lat': 40.71921117375622...",40.719211,-74.008727,10013,NY,Tamarind TriBeCa,v-1574822654,NaN
5,"[{'id': '4bf58dd8d48988d103951735', 'name': 'C...",NaN,NaN,NaN,NaN,NaN,NaN,False,574eab2a498e3f5530cfdbca,100 Church St,...,73,"[100 Church St, New York, NY 10007, United Sta...","[{'label': 'display', 'lat': 40.71228057311295...",40.712281,-74.010129,10007,NY,Indian Beautiful Art (IBA) Crafts PVT Ltd,v-1574822654,NaN
6,"[{'id': '4bf58dd8d48988d10f941735', 'name': 'I...",275521,/delivery_provider_seamless_20180129.png,https://fastly.4sqi.net/img/general/cap/,"[40, 50]",seamless,https://www.seamless.com/menu/a-saffron-thread...,False,510aad1ae4b0bfb2583169b9,98 Chambers St,...,284,"[98 Chambers St (at Church St), New York, NY 1...","[{'label': 'display', 'lat': 40.71482256030567...",40.714823,-74.007689,10007,NY,A Saffron Thread Fresh Indian,v-1574822654,432263795
7,"[{'id': '4bf58dd8d48988d10f941735', 'name': 'I...",79935,/delivery_provider_seamless_20180129.png,https://fastly.4sqi.net/img/general/cap/,"[40, 50]",seamless,https://www.seamless.com/menu/ruchi-120-cedar-...,False,4b12cb6ff964a5204a8e23e3,120 Cedar St,...,422,[120 Cedar St (btwn Greenwich St. & Trinity Pl...,"[{'label': 'display', 'lat': 40.70996196046026...",40.709962,-74.012969,10006,NY,Ruchi Indian Cuisine,v-1574822654,337710444
8,"[{'id': '4bf58dd8d48988d14c941735', 'name': 'W...",299726,/delivery_provider_seamless_20180129.png,https://fastly.4sqi.net/img/general/cap/,"[40, 50]",seamless,https://www.seamless.com/menu/mudville-restaur...,False,45e5c256f964a52046431fe3,126 Chambers St,...,306,[126 Chambers St (btwn W Broadway & Church St)...,"[{'label': 'display', 'lat': 40.71533575723845...",40.715336,-74.008881,10007,NY,Mudville Restaurant & Tap House,v-1574822654,43156651
9,"[{'id': '4bf58dd8d48988d1f5931735', 'name': 'D...",296411,/delivery_provider_seamless_20180129.png,https://fastly.4sqi.net/img/general/cap/,"[40, 50]",seamless,https://www.seamless.com/menu/jing-fong-restau...,False,3fd66200f964a520d5e31ee3,20 Elizabeth St,...,1089,"[20 Elizabeth St (btwn Bayard & Canal St), New...","

In [16]:
filtered_columns = ['name', 'categories'] + [col for col in dataframe.columns if col.startswith('location.')] + ['id']
dataframe_filtered = dataframe.loc[:, filtered_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
dataframe_filtered['categories'] = dataframe_filtered.apply(get_category_type, axis=1)

# clean column names by keeping only last term
dataframe_filtered.columns = [column.split('.')[-1] for column in dataframe_filtered.columns]
#dataframe_filtered

In [15]:
df_indian_rest = dataframe_filtered[dataframe_filtered['categories'] == 'Indian Restaurant']
df_indian_rest

,name,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,postalCode,state,id
0,Bombay's Indian Restaurant,Indian Restaurant,60 Pearl St,US,New York,United States,Broad St.,1017,"[60 Pearl St (Broad St.), New York, NY 10004, ...","[{'label': 'display', 'lat': 40.70356930142193...",40.703569,-74.011141,10004,NY,4c17b29b838020a1fd0de361
4,Tamarind TriBeCa,Indian Restaurant,99 Hudson St,US,New York,United States,at Franklin St,736,"[99 Hudson St (at Franklin St), New York, NY 1...","[{'label': 'display', 'lat': 40.71921117375622...",40.719211,-74.008727,10013,NY,4bbb9dbded7776b0e1ad3e51
6,A Saffron Thread Fresh Indian,Indian Restaurant,98 Chambers St,US,New York,United States,at Church St,284,"[98 Chambers St (at Church St), New York, NY 1...","[{'label': 'display', 'lat': 40.71482256030567...",40.714823,-74.007689,10007,NY,510aad1ae4b0bfb2583169b9
7,Ruchi Indian Cuisine,Indian Restaurant,120 Cedar St,US,New York,United States,btwn Greenwich St. & Trinity Pl.,422,[120 Cedar St (btwn Greenwich St. & Trinity Pl...,"[{'label': 'display', 'lat': 40.70996196046026...",40.709962,-74.012969,10006,NY,4b12cb6ff964a5204a8e23e3
10,Aahar Indian Cuisine,Indian Restaurant,10 Murray St,US,New York,United States,Broadway,140,"[10 Murray St (Broadway), New York, NY 10007, ...","[{'label': 'display', 'lat': 40.71330656515891...",40.713307,-74.007994,10007,NY,575dea4c498e2739e43a27e2
12,Indian Express,Indian Restaurant,18 Murray St,US,New York,United States,Church St.,144,"[18 Murray St (Church St.), New York, NY 10007...","[{'label': 'display', 'lat': 40.71364680217225...",40.713647,-74.008348,10007,NY,4a948b83f964a520f32120e3


#### 5 most famous shopping places near the hotel

In [13]:
search_query = 'Shopping'
radius = 5000
LIMIT = 20
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, hotel_lat, hotel_lon, VERSION, search_query, radius, LIMIT)
results = requests.get(url).json()
venues = results['response']['venues']
dataframe = json_normalize(venues)

In [14]:
filtered_columns = ['name', 'categories'] + [col for col in dataframe.columns if col.startswith('location.')] + ['id']
dataframe_filtered = dataframe.loc[:, filtered_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
dataframe_filtered['categories'] = dataframe_filtered.apply(get_category_type, axis=1)

# clean column names by keeping only last term
dataframe_filtered.columns = [column.split('.')[-1] for column in dataframe_filtered.columns]
dataframe_filtered

,name,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,postalCode,state,id
0,derek loves shopping,Office,NaN,US,NaN,United States,NaN,693,"[New York, United States]","[{'label': 'display', 'lat': 40.7066, 'lng': -...",40.706600,-74.011516,NaN,New York,4b7edcadf964a520090530e3
1,SoHo,Neighborhood,NaN,US,New York,United States,NaN,1307,"[New York, NY 10012, United States]",NaN,40.722603,-74.001238,10012,NY,4c374bf80a71c9b6f2893ec9
2,Shopping Box,Miscellaneous Shop,360 Broome St,US,New York,United States,Mott,1446,"[360 Broome St (Mott), New York, NY 10013, Uni...","[{'label': 'display', 'lat': 40.720156, 'lng':...",40.720156,-73.995424,10013,NY,4c23bb79f1272d7f8ae781c5
3,Mobile Shopping Fall,Event Space,NaN,US,NaN,United States,NaN,928,"[New York, United States]","[{'label': 'display', 'lat': 40.706525, 'lng':...",40.706525,-74.001860,NaN,New York,4e9d77180aaf015a05f89844
4,Mei Bang Shopping Center,Clothing Store,NaN,US,New York,United States,NaN,1048,"[New York, NY, United States]","[{'label': 'display', 'lat': 40.713654, 'lng':...",40.713654,-73.997029,NaN,NY,5252c341498e2b30e7cde0b8
5,Shop Gotham NYC Shopping Tours,Travel Agency,72 Spring St,US,New York,United States,crosby st,1464,"[72 Spring St (crosby st), New York, NY 10012,...","[{'label': 'display', 'lat': 40.72202247464282...",40.722022,-73.997254,10012,NY,5bb55819f05ca8003a062947
6,5th Avenue Shopping Court,Christmas Market,NaN,US,New York,United States,NaN,5241,"[New York, NY 10036, United States]","[{'label': 'display', 'lat': 40.75427654185230...",40.754277,-73.980439,10036,NY,5823afd67b454a0e8888bf57
7,AzokaBD WOMEN'S SHOPPING,Market,19021 Midway Rd,US,Dallas,United States,NaN,1185,"[19021 Midway Rd, Dallas, TX 75287, United Sta...","[{'label': 'display', 'lat': 40.71223177869945...",40.712232,-73.995345,75287,TX,59178da93abcaf1ddc0a6e3f
8,Shopping Rio Sul,General Entertainment,"Rua Lauro Müller, 116",US,Rio de Janeiro,United States,NaN,2631,"[Rua Lauro Müller, 116, Rio de Janeiro, NY 222...","[{'label': 'display', 'lat': 40.7298408, 'lng'...",40.729841,-73.988029,22290-960,NY,4f91b383e4b0935b829b004d
9,Chic NYC Shopping Tour,Miscellaneous Shop,187 Spring St,US,New York,United States,NaN,1503,"[187 Spring St, New York, NY 10012, United Sta...","[{'label': 'display', 'lat': 40.725183, 'lng':...",40.725183,-74.002848,10012,NY,4e342428cc3f2f6e11e649b2


#### Filter categories with market and shopping keywords

In [19]:
df_shopping = dataframe_filtered[dataframe_filtered['categories'] != 'Office']
df_shopping = df_shopping[df_shopping['categories'] != 'Building']
df_shopping = df_shopping[df_shopping['categories'] != 'Boutique']
df_shopping = df_shopping[df_shopping['categories'] != 'Travel Agency']
df_shopping = df_shopping[df_shopping['categories'] != 'Neighborhood']
df_shopping = df_shopping[df_shopping['categories'] != 'General Entertainment']
df_shopping = df_shopping[df_shopping['categories'] != 'Event Space']
df_shopping = df_shopping[df_shopping['categories'] != 'Trail']
df_shopping

,name,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,postalCode,state,id
2,Shopping Box,Miscellaneous Shop,360 Broome St,US,New York,United States,Mott,1446,"[360 Broome St (Mott), New York, NY 10013, Uni...","[{'label': 'display', 'lat': 40.720156, 'lng':...",40.720156,-73.995424,10013,NY,4c23bb79f1272d7f8ae781c5
4,Mei Bang Shopping Center,Clothing Store,NaN,US,New York,United States,NaN,1048,"[New York, NY, United States]","[{'label': 'display', 'lat': 40.713654, 'lng':...",40.713654,-73.997029,NaN,NY,5252c341498e2b30e7cde0b8
6,5th Avenue Shopping Court,Christmas Market,NaN,US,New York,United States,NaN,5241,"[New York, NY 10036, United States]","[{'label': 'display', 'lat': 40.75427654185230...",40.754277,-73.980439,10036,NY,5823afd67b454a0e8888bf57
7,AzokaBD WOMEN'S SHOPPING,Market,19021 Midway Rd,US,Dallas,United States,NaN,1185,"[19021 Midway Rd, Dallas, TX 75287, United Sta...","[{'label': 'display', 'lat': 40.71223177869945...",40.712232,-73.995345,75287,TX,59178da93abcaf1ddc0a6e3f
9,Chic NYC Shopping Tour,Miscellaneous Shop,187 Spring St,US,New York,United States,NaN,1503,"[187 Spring St, New York, NY 10012, United Sta...","[{'label': 'display', 'lat': 40.725183, 'lng':...",40.725183,-74.002848,10012,NY,4e342428cc3f2f6e11e649b2
13,iBraggiotti Jewelry,Jewelry Store,21 Warren St,US,New York,United States,New York,213,"[21 Warren St (New York), New York, NY 10007, ...","[{'label': 'display', 'lat': 40.71410723603152...",40.714107,-74.007786,10007,NY,561795df498e0fb0b2254b38
14,shopping with Sue,Department Store,NaN,US,New York,United States,NaN,4273,"[New York, NY 10001, United States]","[{'label': 'display', 'lat': 40.75020599365234...",40.750206,-73.999123,10001,NY,50bf5af1e4b03b1879d3404f
16,Old Colony Shopping Center,Miscellaneous Shop,NaN,US,Jersey City,United States,NaN,3704,"[Jersey City, NJ 07302, United States]","[{'label': 'display', 'lat': 40.71819945974050...",40.718199,-74.052666,07302,NJ,4b486771f964a520d24c26e3
17,Shoe Shopping,None,NaN,US,NaN,United States,NaN,4909,"[New York, United States]","[{'label': 'display', 'lat': 40.754626, 'lng':...",40.754626,-73.991682,NaN,New York,4d1e252b2eb1f04dbb11ddc1
18,40Th St W Shopping Plaza,Shopping Plaza,NaN,US,New York,United States,NaN,5025,"[New York, NY 10018, United States]","[{'label': 'display', 'lat': 40.75615, 'lng': ...",40.756150,-73.993630,10018,NY,5d34b915e2efbf0008a154c1


#### 5 most famous night clubs near the hotel

In [32]:
search_query = 'Night Club'
radius = 1000
LIMIT = 40
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, hotel_lat, hotel_lon, VERSION, search_query, radius, LIMIT)
results = requests.get(url).json()
venues = results['response']['venues']
dataframe = json_normalize(venues)

In [33]:
filtered_columns = ['name', 'categories'] + [col for col in dataframe.columns if col.startswith('location.')] + ['id']
dataframe_filtered = dataframe.loc[:, filtered_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
dataframe_filtered['categories'] = dataframe_filtered.apply(get_category_type, axis=1)

# clean column names by keeping only last term
dataframe_filtered.columns = [column.split('.')[-1] for column in dataframe_filtered.columns]
dataframe_filtered

,name,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,postalCode,state,id
0,KTU Professional Radio DJ Night Club and Lounge,Nightclub,32 Avenue of the Americas,US,New York,United States,NaN,936,"[32 Avenue of the Americas, New York, NY 10013...","[{'label': 'display', 'lat': 40.720443, 'lng':...",40.720443,-74.005319,10013,NY,4c177cb57bf1d13a4235e881
1,New York Health & Racquet Club,Gym,39 Whitehall St,US,New York,United States,Water Street,1114,"[39 Whitehall St (Water Street), New York, NY ...","[{'label': 'display', 'lat': 40.70293312976079...",40.702933,-74.012749,10004,NY,4acd4277f964a52099cb20e3
2,New York Sports Club,Gym / Fitness Center,217 Broadway,US,New York,United States,at Vesey St,116,"[217 Broadway (at Vesey St), New York, NY 1000...","[{'label': 'display', 'lat': 40.7117534, 'lng'...",40.711753,-74.008596,10007,NY,4aa82d21f964a520d94f20e3
3,Club Remix,Nightclub,27 Park Pl,US,New York,United States,btwn Church St. & Broadway,85,"[27 Park Pl (btwn Church St. & Broadway), New ...","[{'label': 'display', 'lat': 40.71335908709448...",40.713359,-74.009126,10007,NY,4bf19bf2a09076b0e5e529d4
4,SoHo Grand Hotel Club Room,Cocktail Bar,310 W Broadway,US,New York,United States,btwn Grand and Canal,1126,"[310 W Broadway (btwn Grand and Canal), New Yo...","[{'label': 'display', 'lat': 40.72195800761801...",40.721958,-74.004252,10013,NY,4c899f201eafb1f79c847335
5,View of the World Terrace Club,Hotel Bar,144 Washington St,US,New York,United States,at World Center Hotel,441,"[144 Washington St (at World Center Hotel), Ne...","[{'label': 'display', 'lat': 40.71024048755147...",40.710240,-74.013568,10006,NY,4c37d0f493db0f47db922092
6,"Club Quarters Hotel, World Trade Center",Hotel,140 Washington St,US,New York,United States,btwn Albany & Cedar,452,"[140 Washington St (btwn Albany & Cedar), New ...","[{'label': 'display', 'lat': 40.71001751931732...",40.710018,-74.013509,10006,NY,4b181dddf964a52033cd23e3
7,Golden Unicorn Restaurant 麒麟金閣,Dim Sum Restaurant,18 E Broadway,US,New York,United States,at Catherine St,1031,"[18 E Broadway (at Catherine St), New York, NY...","[{'label': 'display', 'lat': 40.71362850464683...",40.713629,-73.997230,10002,NY,3fd66200f964a520ece31ee3
8,"Club Quarters Hotel, Wall Street",Hotel,52 William St,US,New York,United States,NaN,614,"[52 William St, New York, NY 10005, United Sta...","[{'label': 'display', 'lat': 40.70711347511928...",40.707113,-74.008742,10005,NY,4a5e6955f964a52085be1fe3
9,Nexus Club,Social Club,100 Church,US,New York,United States,Barclay,95,"[100 Church (Barclay), New York, NY 10007, Uni...","[{'label': 'display', 'lat': 40.713111, 'lng':...",40.713111,-74.010308,10007,NY,5d1586f72236bd00230c27df


In [29]:
df_night_club = dataframe_filtered[dataframe_filtered['categories'] == 'Nightclub']
df_night_club

,name,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,postalCode,state,id
0,KTU Professional Radio DJ Night Club and Lounge,Nightclub,32 Avenue of the Americas,US,New York,United States,NaN,936,"[32 Avenue of the Americas, New York, NY 10013...","[{'label': 'display', 'lat': 40.720443, 'lng':...",40.720443,-74.005319,10013,NY,4c177cb57bf1d13a4235e881
3,Club Remix,Nightclub,27 Park Pl,US,New York,United States,btwn Church St. & Broadway,85,"[27 Park Pl (btwn Church St. & Broadway), New ...","[{'label': 'display', 'lat': 40.71335908709448...",40.713359,-74.009126,10007,NY,4bf19bf2a09076b0e5e529d4
29,20-20 latin club,Nightclub,NaN,US,New York,United States,NaN,216,"[New York, NY 10013, United States]","[{'label': 'display', 'lat': 40.71419429164759...",40.714194,-74.007878,10013,NY,50f21504e4b09fca7e77d5b4
